In [5]:
#importa bibliotecas necessárias
import mediapipe as mp
import cv2

In [6]:
#inicializa drawing e holistic model
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
fingersError = 0.05
legsError = 0.05

# Funções para detectar poses específicas

def verify_midfinger(landmarks):
    if (landmarks.landmark[12].y < landmarks.landmark[8].y - fingersError and 
        landmarks.landmark[12].y < landmarks.landmark[16].y -fingersError and 
        landmarks.landmark[12].y < landmarks.landmark[20].y - fingersError and
        landmarks.landmark[12].y < landmarks.landmark[9].y):
        return True
    return False

def verify_pose(landmarks):
    right_elbow = landmarks.landmark[14]
    right_wrist = landmarks.landmark[16]
    right_shoulder = landmarks.landmark[12]

    if abs(right_shoulder.y - right_elbow.y) < 0.1 and abs(right_wrist.x - right_elbow.x)<0.1:
        return True
    return False

def verify_raisedlegs(landmarks):
    right_ankle = landmarks.landmark[28]
    left_ankle = landmarks.landmark[27]
    
    if(right_ankle.y < left_ankle.y - legsError):
        return 1
    elif(left_ankle.y < right_ankle.y - legsError):
        return 2
print("Digite '1' para membros inferiores e '2' para superiores")
index = input()

In [14]:
#define a câmera a ser utilizada (0)
cap = cv2.VideoCapture(0)

#instancia o modelo holistic com uma confiança de 50%
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        #captura o frame atual
        ret, frame = cap.read()
    
        #muda as cores do formato BGR para RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        #detecta as landmarks com o modelo holistic e "salva" no objeto results
        results = holistic.process(image)

        #muda de volta para BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        if index == '1':
            landmarks_inferiores = [23, 24, 25, 26, 27, 28, 29, 30, 31, 32
            ]

            conexoes_inferiores = [
                (23, 25), (25, 27), (27, 29), (29, 31), 
                (24, 26), (26, 28), (28, 30), (30, 32),  
                (23, 24)
            ]
            if results.pose_landmarks:
                #desenha os landmarks dos membros inferiores
                for idx in landmarks_inferiores:
                    landmark = results.pose_landmarks.landmark[idx]

                    #converte coordenadas normalizadas em coordenadas de pixels
                    h, w, c = image.shape
                    point = (int(landmark.x * w), int(landmark.y * h))

                    #desenha o ponto (landmark)
                    cv2.circle(image, point, 5, (0, 0, 255), -1)

                #desenha as conexões pré-definidas
                for start_idx, end_idx in conexoes_inferiores:
                    start_landmark = results.pose_landmarks.landmark[start_idx]
                    end_landmark = results.pose_landmarks.landmark[end_idx]

                    #converte coordenadas normalizadas em coordenadas de pixels
                    start_point = (int(start_landmark.x * w), int(start_landmark.y * h))
                    end_point = (int(end_landmark.x * w), int(end_landmark.y * h))

                    #desenha a linha entre os pontos
                    cv2.line(image, start_point, end_point, (0, 255, 0), 2)

                if verify_raisedlegs(results.pose_landmarks) == 1:
                    cv2.putText(image, 'perna direita levantada', (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                if verify_raisedlegs(results.pose_landmarks) == 2:
                    cv2.putText(image, 'perna esquerda levantada', (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                

        elif index == '2':
            #desenha na tela as conexões e as landmarks de: right_hand, left_hand e pose
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            #verifica se as landmarks foram detectadas
            if results.pose_landmarks:
                #verifica se a pose específica determinada pela função "verify_pose" foi detectada
                if verify_pose(results.pose_landmarks):
                    cv2.putText(image, 'pose detectada', (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            if results.right_hand_landmarks:
                if verify_midfinger(results.right_hand_landmarks):
                    cv2.putText(image, 'seja mais educado', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            if results.left_hand_landmarks:
                if verify_midfinger(results.left_hand_landmarks):
                    cv2.putText(image, 'seja mais educado', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                
        cv2.imshow('camera', image)

            #sai do while ao pressionar 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Digite '1' para membros inferiores e '2' para superiores


I0000 00:00:1725569649.303924   63217 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1725569649.305683   74602 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) Graphics (RPL-U)
W0000 00:00:1725569649.357162   74591 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725569649.371902   74595 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725569649.373315   74588 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725569649.373742   74596 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W